In [632]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [634]:
from dukto.pipe import Pipe
from dukto.processor import ColProcessor, MultiColProcessor, Transformer
import pandas as pd
import numpy as np
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer

In [636]:
data  = pd.read_csv('data/ufc.csv', index_col=0)

## ColProcessor
### applies function/s to a column/s  

In [637]:
def convert_foot_to_cm(r):
    if isinstance(r, str) and "'" in r:
        foot, inches = r.split("'")
        inches = int(foot)*12 + int(inches.replace('"', ''))
        return inches*2.54
    return np.nan

def convert_inch_to_cm(r):
    if isinstance(r,str) and '"' in r:
        return int(r.replace('"', '')) * 2.54
    return np.nan

def num_of_num_to_perc(r):
    if isinstance(r,str) and 'of' in r:
        thr, landed = map(int, r.split('of'))
        if landed > 0:
            return thr / landed 
    return np.nan

def pounds_to_kg(r):
    if isinstance(r, str) and 'lbs' in r:
        return int(r.split(' ')[0]) * 0.4535
    return r

In [638]:
single_pipe = [
    ColProcessor(name=['agg_height_first','agg_height_second'], 
                 funcs=[convert_foot_to_cm], funcs_test={"6'2\"":187.96}, suffix='_new'),
    
    ColProcessor(name=['agg_reach_first','agg_reach_second'], 
                 funcs=[convert_inch_to_cm], funcs_test={'70"': 177.80}, suffix='_new'),
    
    ColProcessor(name=['second_total_str', 'first_total_str'], 
                 funcs=[num_of_num_to_perc], suffix='_%%_new', funcs_test={'50 of 100':0.5}),
    
    ColProcessor(name=['agg_dob_first', 'agg_dob_second', 'date_card'], 
                 funcs=[pd.to_datetime]),
    
    ColProcessor(name='agg_weight_first', new_name={"agg_weight_first":'weight_class'}, 
                 funcs=[pounds_to_kg], suffix='_new', drop=True)
]

## MultiColProcessor

## applies a function that takes and returns a dataframe
## this class is used to add columns based on other column/s

In [639]:
def add_ages(df):
    df['first_fighter_age_new'] = df['date_card'] - df['agg_dob_first']
    df['second_fighter_age_new'] = df['date_card'] - df['agg_dob_second']
    return df

def ages_in_years(df):
    df[['first_fighter_age_new', 'second_fighter_age_new']] = df[['first_fighter_age_new', 'second_fighter_age_new']].applymap(lambda x:x/np.timedelta64(1, 'Y'))
    return df

In [640]:
multi_pipe = [
    MultiColProcessor(name=['first_fighter_age_new', 'second_fighter_age_new'], 
                      funcs=[add_ages, ages_in_years]),
             ]

## Transformer

### applies a feature_engine style transformer to a column/s

In [643]:

# add school name so you can encode it

new_cols_func = lambda x: [i for i in x if (('new' in i) and ('weight' not in i))]

trans_pipe  = [
    Transformer(name_from_func=new_cols_func, transformers=[MeanMedianImputer]),
    MultiColProcessor(funcs=[lambda x:x.assign(weight_class_new=x.weight_class_new.astype(str))]),
    Transformer(name=['weight_class_new'], transformers=[CategoricalImputer,CountFrequencyEncoder]),
]

In [622]:
pipeline = single_pipe+multi_pipe+trans_pipe

In [644]:
pipeline

[ColProcessor(agg_height_first, agg_height_second),
 ColProcessor(agg_reach_first, agg_reach_second),
 ColProcessor(second_total_str, first_total_str),
 ColProcessor(agg_dob_first, agg_dob_second, date_card),
 ColProcessor(agg_weight_first),
 MultiColProcessor(first_fighter_age_new, second_fighter_age_new),
 Transformer(),
 MultiColProcessor(),
 Transformer()]

In [624]:
pipe = Pipe(data=data, pipeline=pipeline, run_test_cases=True)

In [625]:
res = pipe.run()

ColProcessor (agg_height_first, agg_height_second) test cases PASSED! 😎
ColProcessor (agg_reach_first, agg_reach_second) test cases PASSED! 😎
ColProcessor (second_total_str, first_total_str) test cases PASSED! 😎
ColProcessor (agg_dob_first, agg_dob_second, date_card) test cases NOT FOUND.
ColProcessor (agg_weight_first)             test cases NOT FOUND.
not implemented yet
not implemented yet


In [647]:
res[[i for i in res.columns if 'new' in i]]

,agg_height_first_new,agg_height_second_new,agg_reach_first_new,agg_reach_second_new,second_total_str_%%_new,first_total_str_%%_new,weight_class_new,first_fighter_age_new,second_fighter_age_new
0,193.04,193.04,213.36,195.58,0.452471,0.629412,454,32.559190,30.119715
1,165.10,175.26,167.64,172.72,0.397059,0.695122,382,31.923996,31.113575
2,195.58,182.88,203.20,187.96,0.666667,0.636364,96,28.063547,26.155226
3,172.72,170.18,177.80,180.34,0.547009,0.391892,614,28.978008,28.509826
4,190.50,177.80,200.66,185.42,0.805195,0.465517,31,35.001403,36.534631
...,...,...,...,...,...,...,...,...,...
5918,154.94,160.02,157.48,162.56,0.469880,0.541667,184,27.858204,33.054751
5919,182.88,185.42,195.58,190.50,0.465753,0.398601,1012,27.223009,28.570060
5920,193.04,177.80,200.66,182.88,0.792880,0.537634,96,31.976016,35.294359
5921,172.72,182.88,175.26,177.80,0.636364,0.500000,897,32.367537,32.186835


In [648]:
data

,agg_dob_first,agg_dob_second,agg_height_first,agg_height_second,agg_reach_first,agg_reach_second,agg_stand_first,agg_stand_second,agg_str_acc_first,agg_str_acc_second,...,date_card,first_fighter_res,first_sig_str_,first_sig_str_percentage,first_total_str,method,second_sig_str_percentage,second_total_str,time,type
0,19-Jul-87,26-Dec-89,"6' 4""","6' 4""","84""","77""",Orthodox,Southpaw,57%,50%,...,8-Feb-20,W,104 of 166,62%,107 of 170,Decision - Unanimous,44%,119 of 263,5:00,belt
1,7-Mar-88,28-Dec-88,"5' 5""","5' 9""","66""","68""",Southpaw,Orthodox,51%,35%,...,8-Feb-20,W,40 of 65,61%,57 of 82,KO/TKO,30%,27 of 68,1:03,belt
2,16-Jan-92,13-Dec-93,"6' 5""","6' 0""","80""","74""",Orthodox,Southpaw,55%,55%,...,8-Feb-20,L,7 of 11,63%,7 of 11,KO/TKO,66%,10 of 15,1:59,NaN
3,16-Feb-91,6-Aug-91,"5' 8""","5' 7""","70""","71""",Orthodox,Orthodox,41%,46%,...,8-Feb-20,L,17 of 60,28%,29 of 74,Decision - Split,48%,64 of 117,5:00,NaN
4,7-Feb-85,28-Jul-83,"6' 3""","5' 10""","79""","73""",Orthodox,Orthodox,50%,39%,...,8-Feb-20,W,20 of 50,40%,27 of 58,Decision - Unanimous,41%,62 of 77,5:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918,10-Aug-93,30-May-88,"5' 1""","5' 3""","62""","64""",Orthodox,Orthodox,67%,46%,...,6/19/2021 0:00,L,23 of 44,52%,26 of 48,TKO - Doctor's Stoppage,45%,39 of 83,5:00,NaN
5919,30-Mar-94,23-Nov-92,"6' 0""","6' 1""","77""","75""",Orthodox,Switch,45%,50%,...,6/19/2021 0:00,W,91 of 260,35%,114 of 286,Decision - Unanimous,45%,68 of 146,5:00,NaN
5920,28-Jun-89,4-Mar-86,"6' 4""","5' 10""","79""","72""",Orthodox,Orthodox,51%,51%,...,6/19/2021 0:00,W,79 of 160,49%,100 of 186,Decision - Split,56%,245 of 309,5:00,NaN
5921,5-Feb-89,12-Apr-89,"5' 8""","6' 0""","69""","70""",Southpaw,Southpaw,37%,42%,...,6/19/2021 0:00,L,1 of 2,50%,1 of 2,KO/TKO,58%,14 of 22,0:37,NaN
